In [ ]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.losses import categorical_crossentropy
from tensorflow.math import add, reduce_sum
from tensorflow import constant, float32
import tensorflow
import numpy as np
#Loss
from keras.backend import argmax, get_value
from tensorflow import make_ndarray, make_tensor_proto 

In [ ]:
def convert_tensor_to_list_of_words(tensor):
  return [word_dict_from_idx[idx] for idx in make_ndarray(make_tensor_proto(argmax(tensor)))]

def custom_loss(y_true, y_pred):
  rhyme_weight = 10
  true = convert_tensor_to_list_of_words(y_true)
  pred = convert_tensor_to_list_of_words(y_pred)
  rhymes_scores = [rhyme_weight*score_rhyme(t, p, enlight_end=1) for t,p in zip(true, pred)]
  s = constant(rhymes_scores, dtype=float32)
  cat_loss = categorical_crossentropy(y_true, y_pred, from_logits=False)
  # tensorflow.print(cat_loss.size)
  return add(s, cat_loss) 

def sample(preds, temperature=0.5):
      # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
#Initialize models
genera_terzina = load_model('/content/weights-55-5.0572.hdf5')
token_terzina = load(open('/content/generaTerzinaTokenizer.pkl', 'rb'))
genera_rima = load_model('/content/model-GeneraRimaWord180Epoch.h5',custom_objects={'custom_loss': custom_loss})
token_rima = load(open('/content/generaRimaWord60.pkl', 'rb'))


In [ ]:
SEQ_LEN = 50
genera_terzina_input = ['<start> vita nostra di cammin del mezzo nel _verse_ oscura selva una per ritrovai mi _verse_ smarrita era via diritta la ché _verse_ <tercet>']
genera_terzina_input = token_terzina.texts_to_sequences(genera_terzina_input)[0]
genera_terzina_input = [0] * (SEQ_LEN - len(genera_terzina_input)) + genera_terzina_input
genera_rima_input = [token_rima.word_index[word] for word in ['vita', 'oscura', 'smarrita']]
print(genera_terzina_input)
print(genera_rima_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 126, 156, 188, 7, 579, 27, 228, 37, 2, 1000, 580, 104, 10, 5666, 18, 2, 2695, 55, 159, 5667, 6, 65, 2, 3]
[155, 999, 2694]


In [ ]:
n_words = 1500
generated = []
first_word = True
for i in range(n_words):
  if first_word:
    genera_rima_input = np.array(genera_rima_input[-3:]).reshape(1,3)
    prediction = np.argmax(genera_rima.predict(genera_rima_input, verbose=0))
    predicted_word = token_rima.index_word[prediction]
    generated.append(predicted_word)
    genera_rima_input = np.append(genera_rima_input,prediction)
    if predicted_word != "<tercet>":
      first_word = False
      genera_terzina_input = np.append(genera_terzina_input, prediction)
  else:
    # Fix the length of the input sequence
    genera_terzina_input = np.array(genera_terzina_input[-50:]).reshape(1,SEQ_LEN)
    prediction = genera_terzina.predict(genera_terzina_input, verbose=0)[0]
    # Sample and encode the output
    val = sample(prediction)
    predicted_word = token_terzina.index_word[val]
    generated.append(predicted_word)
    genera_terzina_input = np.append(genera_terzina_input, val)
    # print("Parola predetta: " + predicted_word)
    # print("Input Corrente: "+ str(genera_terzina_input))
    if predicted_word == '_verse_' or predicted_word == '<tercet>':
      first_word = True

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log


In [ ]:
generated = ['\n' if x=='_verse_' or x=='_end_' or x=='<tercet>' else x for x in generated]
print(generated)
copy = generated.copy()
while copy :
   line = ""
   while copy[0] is not '\n':
    line = copy[0] +" " + line
    copy.pop(0)
   copy.pop(0)
   print(line)

['\n', 'sicura', 'ne', 'lui', 'ma', 'sù', 'come', 'non', '\n', 'punto', 'in', 'sanno', 'che', 'freddo', 'il', 'percosse', '\n', 'schiera', 'son', 'da', 'muro', 'mio', 'al', 'costa', 'di', '\n', '\n', 'vinto', 'dura', 'fu', 'i', 'giace', 'che', 'guisa', 'da', '\n', 'spesso', 'quella', 'a', 'esse', 'tra', 'quella', 'e', '\n', 'santo', 'ne', 'e', 'dispettoso', 'gravi', 'sì', 'ma', '\n', '\n', 'giuso', 'e', 'davante', 'mai', 'lui', 'come', '\n', 'vaso', 'e', 'tardi', 'e', "s'appiglia", 'sé', 'a', "ch'era", '\n', 'scorno', 'e', 'di', 'poeta', 'i', 'nostro', 'miei', 'li', '\n', '\n', 'passo', 'non', 'lor', "'n", 'che', 'prima', 'di', 'parte', '\n', 'molto', 'di', 'carne', 'la', 'a', 'mio', "l'ha", 'che', '\n', 'trapasso', 'al', 'retro', 'un', 'e', 'piangere', 'per', 'luce', 'di', '\n', '\n', 'sparto', 'terra', 'più', 'al', 'monte', 'al', 'retro', 'di', '\n', 'prande', 'poi', 'e', 'dal', 'mondo', 'il', 'al', 'come', '\n', 'quarto', 'quel', 'come', 'cara', 'in', 'retro', 'di', '\n', '\n', 'spl